## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
# from config.py import weather_api_key

# Configure gmaps API key
from config import g_key

gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Vila Velha,-20.3297,-40.2925,82.35,64,0,14.97,BR,2022-07-17 14:09:07
1,1,Rikitea,-23.1203,-134.9692,73.67,77,100,16.84,PF,2022-07-17 14:18:41
2,2,Kapaa,22.0752,-159.3190,80.58,82,75,17.27,US,2022-07-17 14:18:41
3,3,Arraial Do Cabo,-22.9661,-42.0278,82.36,54,0,9.22,BR,2022-07-17 14:18:41
4,4,Barrow,71.2906,-156.7887,35.62,87,100,16.11,US,2022-07-17 14:09:36


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 78
What is the maximum temperature you would like for your trip? 87


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Vila Velha,-20.3297,-40.2925,82.35,64,0,14.97,BR,2022-07-17 14:09:07
2,2,Kapaa,22.0752,-159.3190,80.58,82,75,17.27,US,2022-07-17 14:18:41
3,3,Arraial Do Cabo,-22.9661,-42.0278,82.36,54,0,9.22,BR,2022-07-17 14:18:41
6,6,Georgetown,5.4112,100.3354,85.95,77,20,1.14,MY,2022-07-17 14:09:34
7,7,Sao Joao Da Barra,-21.6403,-41.0511,84.29,56,100,10.92,BR,2022-07-17 14:12:10
16,16,Hilo,19.7297,-155.0900,80.28,97,100,8.05,US,2022-07-17 14:10:56
19,19,Zaraza,9.3503,-65.3245,85.10,63,30,6.80,VE,2022-07-17 14:18:43
20,20,Sao Jose Da Coroa Grande,-8.8978,-35.1478,79.34,70,4,14.76,BR,2022-07-17 14:18:44
21,21,Albany,42.6001,-73.9662,81.63,67,39,1.01,US,2022-07-17 14:13:52
44,44,Skegness,53.1436,0.3363,86.07,52,100,9.84,GB,2022-07-17 14:18:48


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 191 entries, 0 to 681
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     191 non-null    int64  
 1   City        191 non-null    object 
 2   Lat         191 non-null    float64
 3   Lng         191 non-null    float64
 4   Max Temp    191 non-null    float64
 5   Humidity    191 non-null    int64  
 6   Cloudiness  191 non-null    int64  
 7   Wind Speed  191 non-null    float64
 8   Country     191 non-null    object 
 9   Date        191 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 16.4+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Vila Velha,BR,82.35,-20.3297,-40.2925,
2,Kapaa,US,80.58,22.0752,-159.3190,
3,Arraial Do Cabo,BR,82.36,-22.9661,-42.0278,
6,Georgetown,MY,85.95,5.4112,100.3354,
7,Sao Joao Da Barra,BR,84.29,-21.6403,-41.0511,
16,Hilo,US,80.28,19.7297,-155.0900,
19,Zaraza,VE,85.10,9.3503,-65.3245,
20,Sao Jose Da Coroa Grande,BR,79.34,-8.8978,-35.1478,
21,Albany,US,81.63,42.6001,-73.9662,
44,Skegness,GB,86.07,53.1436,0.3363,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping the city.")
        
# hotel_df.loc[index, "Hotel Name"] = np.nan       

Hotel not found... skipping the city.
Hotel not found... skipping the city.
Hotel not found... skipping the city.
Hotel not found... skipping the city.
Hotel not found... skipping the city.
Hotel not found... skipping the city.
Hotel not found... skipping the city.
Hotel not found... skipping the city.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Vila Velha,BR,82.35,-20.3297,-40.2925,Hotel Vitória Palace
2,Kapaa,US,80.58,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,Arraial Do Cabo,BR,82.36,-22.9661,-42.0278,Pousada Porto Praia
6,Georgetown,MY,85.95,5.4112,100.3354,Cititel Penang
7,Sao Joao Da Barra,BR,84.29,-21.6403,-41.0511,Pousada Porto De Canoas


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))